In [74]:
import mysql.connector as connection
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# check if the connection is established
mydb = connection.connect(host="localhost", user="root", passwd='1321',use_pure=True) #start connection
print(mydb.is_connected())
mydb.close() #close connection

True


## Checking the list of Databases 

In [17]:
mydb = connection.connect(host="localhost",user="root", passwd="1321",use_pure=True)
# check if the connection is established
query = "SHOW DATABASES"
cursor = mydb.cursor() #create a cursor to execute queries
cursor.execute(query)
print(cursor.fetchall())

[('bootcamp',), ('countries',), ('demo_trigger',), ('ecommerce1',), ('employees',), ('information_schema',), ('innomatics',), ('learndb',), ('learndb2',), ('library',), ('location',), ('mobile_phones',), ('mysql',), ('netflix',), ('ouickcommerce',), ('performance_schema',), ('phones',), ('pointstable',), ('productdb',), ('quickcommerce',), ('sakila',), ('student',), ('students',), ('sys',), ('whiskey_analysis',), ('whiskey_analysis_py',), ('wiskey_analysis',), ('world',), ('world_whiskey_analysis',)]


## Create Database

In [15]:
query = "CREATE DATABASE whiskey_analysis_py;"
cursor = mydb.cursor()
cursor.execute(query)
print("Database Created!!")
mydb.close()

Database Created!!


## Creating Table whiskey

In [19]:
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

query = """
CREATE TABLE whiskey
(
brand varchar(255) , 
varieties varchar(255) , 
type varchar(255) , 
centiliter int ,
year varchar(255) , 
abv_percent int , 
price_rupees int ,
abv_category varchar(255) ,
mili_liter int ,
country varchar(255)
);
"""
cursor = mydb.cursor()
cursor.execute(query)
print("whiskey table created.")
mydb.close()


whiskey table created.


## Inserting Data Using CSV file 

In [21]:
df = pd.read_csv('Whiskey_cleaned_data_final.csv')
df.head()

,Brand,Varieties,Type,Centi_liter,Year,ABV_Percent,Price_Rupees,ABV_Category,Mili_liter,Country
0,Angel's Envy,Straight Port Cask Finish Bourbon,Bourbon,70,NAS,3,6810,Low,700,United States
1,Angel's Envy,Manhattan Bundle - Martini Rubino Vermouth & S...,Bourbon,70,NAS,3,9082,Low,700,United States
2,Buffalo Trace,Traveller Bourbon Whiskey (Chris,Bourbon,70,NAS,45,3630,Medium,700,United States
3,Buffalo Trace,Kosher Wheat Recipe Bourbon,Bourbon,75,NAS,47,4993,High,750,United States
4,Elijah Craig,Barrel Proof Kentucky Straight Bourbon,Barrel,70,NAS,8,14194,Low,700,Scotland


In [23]:
import mysql.connector as connection
import pandas as pd

# Step 1: Load and clean the data
df = pd.read_csv("Whiskey_cleaned_data_final.csv")
df.columns = df.columns.str.strip().str.lower()

# Step 2: Convert numeric fields safely
for col in ['price_rupees', 'centi_liter', 'abv_percent', 'mili_liter']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 3: Replace NaNs with None
df = df.where(pd.notnull(df), None)

# Step 4: Connect to the database
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)
cursor = mydb.cursor()

# Step 5: Insert row-by-row
for _, row in df.iterrows():
    sql = """
    INSERT INTO whiskey (brand, varieties, type, centiliter, year, abv_percent, abv_category, price_rupees, mili_liter, country)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        row['brand'],
        row['varieties'],
        row['type'],
        row['centi_liter'],
        row['year'],
        row['abv_percent'],
        row['abv_category'],
        row['price_rupees'],
        row['mili_liter'],
        row['country']
    )
    cursor.execute(sql, values)

# Finalize
mydb.commit()
cursor.close()
mydb.close()

print("Data inserted successfully.")


Data inserted successfully.


## Questions For Extracting Insights

### `Rank whiskies within each country based on price (Top 5 highest first)`

In [29]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """SELECT 
    brand, 
    country, 
    price_rupees
FROM (
    SELECT 
        brand,
        country,
        price_rupees,
        DENSE_RANK() OVER (
            PARTITION BY country 
            ORDER BY price_rupees DESC
        ) AS price_rank_country
    FROM 
        whiskey
) AS ranked_whiskey
WHERE 
    price_rank_country = 1
    ORDER BY price_rupees DESC
    LIMIT 5;"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Brands' , 'Country' , 'Price']])
print(df)

# Close
cursor.close()
mydb.close()


       Brands        Country    Price
0    Midleton        England  5452794
1    Yamazaki          Japan  1022394
2    Yamazaki  United States   624800
3  Eagle Rare       Kentucky   477114
4   Bushmills        Ireland   215834


In [31]:
df

,Brands,Country,Price
0,Midleton,England,5452794
1,Yamazaki,Japan,1022394
2,Yamazaki,United States,624800
3,Eagle Rare,Kentucky,477114
4,Bushmills,Ireland,215834


### `Retrieve all whiskies with an ABV greater than 50%.`

In [70]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """
SELECT brand , abv_percent
FROM whiskey 
WHERE abv_percent > 50;
"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Brand' , 'Abv%']])
print(df)

# Close
cursor.close()
mydb.close()


                  Brand Abv%
0           Yellowstone   54
1  William Larue Weller   57
2              Stagg Jr   95
3           Macaloney’s   57
4              Yamazaki   56
5          The Irishman   54
6              Starward   52
7              Starward   56


In [72]:
df

,Brand,Abv%
0,Yellowstone,54
1,William Larue Weller,57
2,Stagg Jr,95
3,Macaloney’s,57
4,Yamazaki,56
5,The Irishman,54
6,Starward,52
7,Starward,56


### `Calculate running total of whisky prices by brand`

In [46]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """
SELECT brand ,
price_rupees , 
SUM(price_rupees) OVER(PARTITION BY brand ORDER BY price_rupees DESC) running_total
FROM whiskey;
"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Brand' , 'Price' , 'Running_Total']])
print(df)

# Close
cursor.close()
mydb.close()


           Brand  Price Running_Total
0     Aber Falls   7946          7946
1     Aber Falls   6810         14756
2     Aber Falls   6697         21453
3     Aber Falls   2834         24287
4          Amrut  11014         11014
..           ...    ...           ...
437     Yamazaki   9650       4710760
438     Yamazaki   6810       4717570
439  Yellow Spot   9082          9082
440  Yellowstone   7719          7719
441  Yellowstone   6810         14529

[442 rows x 3 columns]


In [48]:
df

,Brand,Price,Running_Total
0,Aber Falls,7946,7946
1,Aber Falls,6810,14756
2,Aber Falls,6697,21453
3,Aber Falls,2834,24287
4,Amrut,11014,11014
...,...,...,...
437,Yamazaki,9650,4710760
438,Yamazaki,6810,4717570
439,Yellow Spot,9082,9082
440,Yellowstone,7719,7719


### `Count the number of whiskies for each brand.`

In [51]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """
SELECT brand , COUNT(*) as Number_of_whiskeys
FROM whiskey 
GROUP BY brand;
"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Brand' , 'No_of_whiskeys']])
print(df)

# Close
cursor.close()
mydb.close()

            Brand No_of_whiskeys
0    Angel's Envy              2
1   Buffalo Trace              8
2    Elijah Craig              3
3    Maker's Mark              2
4    Ben Holladay              1
..            ...            ...
95           Kyro              3
96       Starward             14
97       Shizuoka              5
98      Waterford             20
99        Teeling             20

[100 rows x 2 columns]


In [53]:
df

,Brand,No_of_whiskeys
0,Angel's Envy,2
1,Buffalo Trace,8
2,Elijah Craig,3
3,Maker's Mark,2
4,Ben Holladay,1
...,...,...
95,Kyro,3
96,Starward,14
97,Shizuoka,5
98,Waterford,20


### `Show each whisky along with the average ABV of its type`

In [56]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """
SELECT type , 
AVG(abv_percent) as avg_abv
FROM
whiskey
GROUP BY type
ORDER BY avg_abv DESC;
"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Type' , 'Avg_ABV']])
print(df)

# Close
cursor.close()
mydb.close()

          Type  Avg_ABV
0   Rum Finish  50.0000
1      Suntory  47.0000
2     American  43.1667
3      Harmony  43.0000
4   Australian  42.9286
5     Canadian  42.2500
6    Tennessee  41.7857
7    Formula 1  41.0000
8        Welsh  38.5789
9     Japanese  38.4186
10       Irish  37.5203
11      Indian  35.9000
12   Very Rare  32.5238
13      Barrel  31.1935
14     English  27.2308
15         Rye  27.1034
16     Bourbon  22.4000
17    Kentucky  21.9091
18     Swedish  12.1739
19       Eagle   5.0000


In [58]:
df

,Type,Avg_ABV
0,Rum Finish,50.0000
1,Suntory,47.0000
2,American,43.1667
3,Harmony,43.0000
4,Australian,42.9286
5,Canadian,42.2500
6,Tennessee,41.7857
7,Formula 1,41.0000
8,Welsh,38.5789
9,Japanese,38.4186


### `Find the difference in price between each whisky and the previous one (by brand)`

In [61]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """
SELECT brand , 
price_rupees , 
LAG(price_rupees) OVER(PARTITION BY brand ORDER BY price_rupees ) previous_one , 
price_rupees - LAG(price_rupees) OVER(PARTITION BY brand ORDER BY price_rupees) price_diff
FROM 
whiskey;
"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Brand' , 'Price' , 'Previous_Price' , 'Price_diff']])
print(df)

# Close
cursor.close()
mydb.close()

           Brand    Price Previous_Price Price_diff
0     Aber Falls     2834            NaN        NaN
1     Aber Falls     6697         2834.0     3863.0
2     Aber Falls     6810         6697.0      113.0
3     Aber Falls     7946         6810.0     1136.0
4          Amrut     5106            NaN        NaN
..           ...      ...            ...        ...
437     Yamazaki   795194       624800.0   170394.0
438     Yamazaki  1022394       795194.0   227200.0
439  Yellow Spot     9082            NaN        NaN
440  Yellowstone     6810            NaN        NaN
441  Yellowstone     7719         6810.0      909.0

[442 rows x 4 columns]


In [63]:
df

,Brand,Price,Previous_Price,Price_diff
0,Aber Falls,2834,NaN,NaN
1,Aber Falls,6697,2834.0,3863.0
2,Aber Falls,6810,6697.0,113.0
3,Aber Falls,7946,6810.0,1136.0
4,Amrut,5106,NaN,NaN
...,...,...,...,...
437,Yamazaki,795194,624800.0,170394.0
438,Yamazaki,1022394,795194.0,227200.0
439,Yellow Spot,9082,NaN,NaN
440,Yellowstone,6810,NaN,NaN


### `For each type, show the whisky with the highest ABV in each country`

In [66]:
import mysql.connector as connection
import pandas as pd

# Connect to MySQL
mydb = connection.connect(
    host="localhost",
    database="whiskey_analysis_py",  
    user="root",
    passwd="1321",
    use_pure=True
)

cursor = mydb.cursor()

# Execute query
query = """
WITH RankedWhiskies AS (
    SELECT 
        brand,
        type,
        country,
        abv_percent,
        DENSE_RANK() OVER (
            PARTITION BY type, country 
            ORDER BY abv_percent DESC
        ) AS abv_rank
    FROM 
        whiskey
)
SELECT 
    type,
    COUNT(*) AS top_whiskies_count
FROM 
    RankedWhiskies
WHERE 
    abv_rank = 1
GROUP BY 
    type
ORDER BY top_whiskies_count DESC;
"""
cursor.execute(query)

# results
results = cursor.fetchall()

# DataFrame
df = pd.DataFrame(results, columns=[['Type' , 'Top_Whiskey_Count']])
print(df)

# Close
cursor.close()
mydb.close()

          Type Top_Whiskey_Count
0      Harmony                 9
1     Japanese                 9
2    Tennessee                 5
3          Rye                 4
4    Formula 1                 2
5    Very Rare                 2
6        Welsh                 2
7     American                 1
8   Australian                 1
9       Barrel                 1
10     Bourbon                 1
11    Canadian                 1
12       Eagle                 1
13     English                 1
14      Indian                 1
15       Irish                 1
16    Kentucky                 1
17  Rum Finish                 1
18     Suntory                 1
19     Swedish                 1


In [68]:
df

,Type,Top_Whiskey_Count
0,Harmony,9
1,Japanese,9
2,Tennessee,5
3,Rye,4
4,Formula 1,2
5,Very Rare,2
6,Welsh,2
7,American,1
8,Australian,1
9,Barrel,1
